# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. HARSHA K
2. SANTOSH V BHAT
3. A N SRINIVASAN
4. YASH LODHA

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

**PEAS Environment for the Problem:**

***Performance Measure:***
> _Minimized communication lines with un-interrupted network._

***Environment:***
> _Shared and distributed servers placed at various locations in the city. Communication lines that connects all these servers._

***Actuators (O/P):***
> _The agent's actions responsible for displaying optimum path between servers by activating or deactivating communication lines._

***Sensors (I/P):***
> _Sensors in this environment provide information about the current network topology at given instance, also the start and goal node of the distributed server._

Design the agent as PSA Agent(Problem Solving Agent) 
Clear Initial data structures to define the graph and variable declarations is expected 
IMPORTANT: Write distinct code block as below

In [ ]:
#Code Block : Set Initial State (Must handle dynamic inputs)

from enum import Enum, auto
import re
import pprint

class Server(Enum):
    A = 0
    B = auto()
    C = auto()
    D = auto()
    E = auto()
    F = auto()
    G = auto()
    H = auto()

    def __str__(self):
        return self.name

# Graph is constructed statically as per given topology
graph = {
    Server.A: [(Server.B, 15), (Server.C, 10), (Server.D, 17), (Server.G, 5)],
    Server.B: [(Server.A, 15), (Server.D, 12)],
    Server.C: [(Server.A, 10), (Server.G, 7)],
    Server.D: [(Server.A, 17), (Server.B, 12), (Server.E, 2), (Server.F, 10), (Server.H, 4)],
    Server.E: [(Server.D, 2)],
    Server.F: [(Server.D, 10), (Server.H, 11)],
    Server.G: [(Server.A, 5), (Server.C, 7), (Server.H, 25)],
    Server.H: [(Server.D, 4), (Server.F, 11), (Server.G, 25)],
}

# Function to get graph items dynamically from user
def get_graph_items():
    graph = {}
    server_regex = re.compile(r'[A-H]')
    cost_regex = re.compile(r'\d+')
    while True:
        server_input = input("Enter the server (A-H) or 'done' to finish: ")
        if server_input == 'done':
            break
        if not server_regex.match(server_input):
            print("Invalid server input. Please enter a valid server (A-H).")
            continue
        server = Server[server_input]
        connections = []
        while True:
            connection_input = input(f"Enter a connected server for {server_input} or 'done' to finish: ")
            if connection_input == 'done':
                break
            if not server_regex.match(connection_input):
                print("Invalid server input. Please enter a valid server (A-H).")
                continue
            connection_server = Server[connection_input]
            cost_input = input(f"Enter the cost for the connection between {server_input} and {connection_input}: ")
            if not cost_regex.match(cost_input):
                print("Invalid cost input. Please enter a valid cost (integer).")
                continue
            cost = int(cost_input)
            connections.append((connection_server, cost))
        graph[server] = connections
    return graph

# Get user input to determine whether to use dynamic graph or default one
use_dynamic_graph = input("Do you want to use a dynamic graph? (y/n): ")
if use_dynamic_graph.lower() == 'y':
    graph = get_graph_items()
    print("Graph:", graph)
else:
    print("Using default graph.")

# Get the graph items from the user
print("Graph:")
pprint.pprint(graph)

class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0, f=0):
        self.state = state # current state
        self.parent = parent # parent node
        self.action = action # action taken to reach this node
        self.path_cost = path_cost # path cost to reach this node g(n)
        self.f = f # f(n) = g(n) + h(n). Used only on RBFS algorithm

    def __repr__(self):
        return "<Node {}>".format(self.state)

# Get and validate the Node from user input
def get_input_node(message):
    # Valid inputs are A, B, C, D, E, F, G, H.
    server_input = input(message)
    # Convert the input string to the corresponding enum member
    try:
        server = Server[server_input]
        print("Server accepted:", {server.name})
    except KeyError:
        print("Error: ", {server_input}, " is not a valid server (A-H), try again...")
        server = get_input_node(message)
    return server

# Initial/S0/Agent-in/start state
start_state = get_input_node("Enter the node where agent is present: ")
goal_state = get_input_node("Enter the node where agent should reach/goal node: ")

print("\nStart state: ", start_state)
print("Goal state: ", goal_state)

In [ ]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)

import numpy as np

# # Below transition cost matrix is derived from given topology/graph
# transition_cost = [
#     [0, 15, 10, 17, np.inf, np.inf, 5, np.inf],
#     [15, 0, np.inf, 12, np.inf, np.inf, np.inf, np.inf],
#     [10, np.inf, 0, np.inf, np.inf, np.inf, 7, np.inf],
#     [17, 12, np.inf, 0, 2, 10, np.inf, 4],
#     [np.inf, np.inf, np.inf, 2, 0, np.inf, np.inf, np.inf],
#     [np.inf, np.inf, np.inf, 10, np.inf, 0, np.inf, 11],
#     [5, np.inf, 7, np.inf, np.inf, np.inf, 0, 25],
#     [np.inf, np.inf, np.inf, 4, np.inf, 11, 25, 0]
# ]
# transition_cost_matrix = np.matrix(transition_cost)

# Derive transition cost matrix from given graph
def get_transition_cost_matrix(graph, servers):
    transition_cost = np.full((len(servers), len(servers)), np.inf)
    for server, connections in graph.items():
        for connection, cost in connections:
            transition_cost[server.value][connection.value] = cost
    for i in range(len(servers)):
        transition_cost[i][i] = 0
    return transition_cost

transition_cost_matrix = get_transition_cost_matrix(graph, list(Server))

print("Transition cost matrix:")
print(transition_cost_matrix)

In [ ]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented

from collections import deque

# def step_cost(state, action):
#     # Using graph
#     # Return the cost of taking the given action in the given state
#     for neighbor, cost in graph[state]:
#         if neighbor == action:
#             return cost
#     return 0

# def actions(state):
#     # Using graph
#     # Return a list of possible actions from the given state
#     actions = []
#     for neighbor, _ in graph[state]:
#         actions.append(neighbor)
#     print("Nodes generated from ", state, ":", actions)
#     return actions

# def result(state, action):
#     # Using graph
#     # Return the state that results from taking the given action in the given state
#     for neighbor, _ in graph[state]:
#         if neighbor == action:
#             return neighbor
#     return state

def step_cost(state, action):
    # Using transition matrix
    # Return the cost of taking the given action in the given state
    return transition_cost_matrix[state.value, action.value]

def actions(state):
    # Using transition matrix
    # Return a list of possible actions from the given state
    row = transition_cost_matrix[state.value]
    indices = np.where(row != np.inf)[0]
    actions = [Server(index) for index in indices]
    return actions

def result(state, action):
    # Using transition matrix
    # Return the state that results from taking the given action in the given state
    if transition_cost_matrix[state.value, action.value] != np.inf:
        return action
    return state

def transition_model(parent, action):
    # Return the state that results from taking the given action in the given state
    state = result(parent.state, action)
    path_cost = parent.path_cost + step_cost(parent.state, action)
    return Node(state, parent, action, path_cost)

def solution(node):
    # Return the sequence of actions to go from the root to given node.
    action_list = deque()
    path_cost = node.path_cost
    while node.parent is not None:
        action_list.appendleft(node.action.name)
        node = node.parent
    # Finally add the start state to the actions list
    action_list.appendleft(node.state.name)
    return {'path': list(action_list), 'path_cost': path_cost}

# Print the solution and path cost
def display_result(input):
    if input['status'] == False:
        print("No solution found")
    else:
        path = input['solution']['path']
        print("\nPath: " + " -> ".join(path))
        print("Path Cost: ", input['solution']['path_cost'])


In [ ]:
#Code block : Write function to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented

def goal_test(state, display=True):
    res = state == goal_state
    if display:
        print(f"Goal Test {state} : {'Pass' if res else 'Fail'}")
    return res

In [ ]:
# For heuristic design, consider all the possible paths between any arbitrary node n to the goal node.
# The average of the total transmission cost across all these paths is the heuristic value h(n)

import numpy as np
from collections import deque

def calculate_heuristic(node):
    # Calculate the heuristic value for the given node
    total_cost = 0
    num_paths = 0

    # Use Depth-First Search (DFS) to find all paths from the current node to the goal node
    stack = deque([(node.state, [node.state])])
    while stack:
        current_node, path = stack.pop()
        if current_node == goal_state:
            # Calculate the path cost and update the total cost
            path_cost = calculate_path_cost(path)
            total_cost += path_cost
            num_paths += 1
        else:
            # Explore the neighbors of the current node
            for neighbor, _ in graph[current_node]:
                if neighbor not in path:
                    stack.append((neighbor, path + [neighbor]))

    # Calculate the average cost across all paths
    if num_paths > 0:
        heuristic_value = total_cost / num_paths
    else:
        heuristic_value = 0

    return heuristic_value

def calculate_path_cost(path):
    # Calculate the total cost of a given path
    total_cost = 0
    for i in range(len(path) - 1):
        current_node = path[i]
        next_node = path[i + 1]
        total_cost += transition_cost_matrix[current_node.value, next_node.value]
    return total_cost

# Calculate heuristic values
h_values = {server: calculate_heuristic(Node(server)) for server in Server}
for h_value in h_values:
    print(f"Node: {h_value}, Heuristic: {h_values[h_value]}")

### 2.	Definition of Algorithm 1 (UNINFORMED SEARCH - BREADTH FIRST SEARCH)

In [ ]:
#Code Block : Function for algorithm 1 implementation

# TODO: Note 1, point 4: "Include code in your implementation to calculate the space complexity and time complexity and print the same."

def breadth_first_search(node):
    # Check if the initial node is the goal
    if goal_test(node.state):
        return {'status': True, 'solution': solution(node)}
    
    # Initialize the frontier with the initial node
    frontier = deque([node])

    # Initialize the explored set to keep track of visited states (to avoid looping problem)
    explored = set()

    # Loop until the frontier is empty
    while frontier:
        print("\nFrontier: ", [n.state.name for n in frontier])
        node = frontier.popleft()
        explored.add(node.state)

        # Iterate over all possible actions from the current state
        for action in actions(node.state):
            # Generate the child node by applying the action
            child = transition_model(node, action)

            if child.state not in explored and child not in frontier:
                # Check if the child node is the goal
                if goal_test(child.state):
                    return {'status': True, 'solution': solution(child)}
                
                # Add the child node to the frontier
                frontier.append(child)

    # If no solution is found, return an empty path
    return {'status': False, 'solution': None}

### 3.	Definition of Algorithm 2 (INFORMED SEARCH - Recursive Best First Search)

In [ ]:
# Code Block : Function for algorithm 2 implementation

# TODO: Note 1, point 4: "Include code in your implementation to calculate the space complexity and time complexity and print the same."

def recursive_best_first_search(node, f_limit=float('inf'), explored=set(), cost=0):
    # Check if the initial node is the goal
    print()
    if goal_test(node.state):
        return {'status': True, 'solution': solution(node)}

    # Adding node to explored set to avoid looping problem
    explored.add(node.state)

    # Generate successors of the current node
    successors = []
    for action in actions(node.state):
        child = transition_model(node, action)
        # To avoid looping problem, only add child if it hasn't been explored
        if child.state not in explored:
            successors.append(child)

    # Printing the successors
    print(f"Successors of {node.state}: " + " | ".join(f"{s.state}" for s in successors))

    # If there are no successors, return failure with infinite cost
    if not successors:
        explored.remove(node.state)
        return {'status': False, 'solution': None}

    # Update f-values for all successors
    for s in successors:
        s.f = s.path_cost + h_values[s.state]

    while successors:
        # Find the best and second-best successors using f_value
        best, alternative = None, None
        best_value, alternative_value = float('inf'), float('inf')
        for s in successors:
            f_value = s.f
            if f_value < best_value:
                alternative, alternative_value = best, best_value
                best, best_value = s, f_value
            elif f_value < alternative_value:
                alternative, alternative_value = s, f_value

        print("Best path node: ", best.state)
        print("Second-best path node: ", alternative.state if alternative else "None")

        # Call recursively with the best successor and new f_limit
        output = recursive_best_first_search(best, min(f_limit, alternative_value), explored, best.path_cost)

        # If the recursive call returns a solution, send it upwards
        if output['status'] != False:
            return output

        # Remove the best node from successors
        successors.remove(best)

    return {'status': False, 'solution': None}


### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question. 

In [ ]:
#Code Block : Function & call to get inputs (start/end state)
print("start_state and goal_state is already set in the initial state block")

### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [ ]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))

# Invoke the breadth first search algorithm
path = breadth_first_search(Node(start_state, None, None, 0))
display_result(path)

# TODO: Note 1, point 2: "Compare to interpret the results in terms of the algorithm working, performance & shortest path"
# TODO: Note 1, point 4: "Include code in your implementation to calculate the space complexity and time complexity and print the same."

In [ ]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))

# Invoke the recursive best first search algorithm
path = recursive_best_first_search(Node(start_state, None, None, 0), f_limit=float('inf'), explored=set(), cost=0)
display_result(path)

# TODO: Note 1, point 2: "Compare to interpret the results in terms of the algorithm working, performance & shortest path"
# TODO: Note 1, point 4: "Include code in your implementation to calculate the space complexity and time complexity and print the same."


### 5.	Comparitive Analysis

In [ ]:
#Code Block : Print the Time & Space complexity of algorithm 1 

# TODO: Testing
# TODO: Correct the formula for the non leaf edges

# V = len(Server)  # Number of vertices
# E = np.count_nonzero(transition_cost_matrix != np.inf)  # Number of edges

# time_complexity = f"O({V} + {E})"
# space_complexity = f"O({V})"

# print(f"Time Complexity: {time_complexity}")
# print(f"Space Complexity: {space_complexity}")


import math

def calculate_depth(n, b):
    if b == 1:
        # If branching factor is 1, the tree is a linear chain
        return n - 1
    
    return (math.log(n*b -n + 1) - math.log(b)) - 1

V = len(Server)  # Number of vertices
E = np.count_nonzero(transition_cost_matrix != np.inf) // 2  # Number of edges
non_leaf_nodes = sum(1 for node_edges in graph.values() if node_edges)        
b = E // non_leaf_nodes  # Approximate branching factor (edges per vertex)
    
n = V  # Number of nodes
d = calculate_depth(n, b)

time_complexity = f"O({b}^{d})"
space_complexity = f"O({b}^{d})"

print(f"Time Complexity: {time_complexity}")
print(f"Space Complexity: {space_complexity}")
print(f"Depth of the tree: {d:.2f}")

In [ ]:
#Code Block : Print the Time & Space complexity of algorithm 2

### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : _______________________________________________

________________________________________________________

_________________________________________________________